In [25]:
import os
import time
import langchain
from langchain_openai import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [5]:
os.environ['OPENAI_API_KEY'] = 'ENTER-YOUR-API-KEY-FOR-VECTORIZATION'

In [6]:
llm = OpenAI(temperature=0.9, max_tokens=500)

In [34]:
urls = [
    "https://finance.yahoo.com/world-indices/",
     "https://www.geo.tv/latest/537434-ex-cjp-jilani-refuses-to-head-commission-formed-to-probe-ihc-judges-letter",
    "https://www.geo.tv/latest/537318-petrol-price-goes-up-by-rs966-per-litre",
    "https://www.geo.tv/latest/537379-british-museum-probed-over-concealed-ethiopian-artefacts-held-for-150-years",
    "https://www.geo.tv/latest/537410-senate-elections-2024-ecp-finalises-preparations-for-polling-tomorrow",
    "https://www.binance.com/en/markets/overview",
]

loaders = UnstructuredURLLoader(urls=urls)

data = loaders.load()

print("Number of fetched URLs:", len(data))

Number of fetched URLs: 6


In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    separators=['\n\n', '\n', '.', ','],
)
docs = text_splitter.split_documents(data)
len(docs)

33

In [41]:
docs[30]

Document(page_content='According to the Election Commission of Pakistan (ECP), the elections aim to fill the seats left vacant by half of the current senators who retired on March 12. Out of the 52 senators who completed their six-year terms and retired, elections are being held for 48 seats.\n\nThis is because four seats were abolished following the merger of the former federally administered tribal areas (FATA) regions into the Khyber-Pakhtunkhwa province.\n\nConsequently, the Senate’s total number of seats has decreased from 100 to 96. Recently, the ECP released a detailed code of conduct for parties and candidates ahead of the Senate elections.\n\nThe code aims to guarantee transparency, integrity, and fairness throughout the electoral process. The code of conduct imposes stringent guidelines on political parties and candidates involved in the elections.', metadata={'source': 'https://www.geo.tv/latest/537410-senate-elections-2024-ecp-finalises-preparations-for-polling-tomorrow'})

In [20]:
embeddings = OpenAIEmbeddings()

D:\Knowledge Streams\DS\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [21]:
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [30]:
vectorindex_openai.save_local("LLM_MODEL")

In [31]:
model = FAISS.load_local("LLM_MODEL", embeddings, allow_dangerous_deserialization=True)

In [32]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever = vectorindex_openai.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x000001A1948157F0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001A18D853BF0>, temperature=0.9, max_tokens=500, openai_api_key=SecretStr('**********'), openai_proxy='')), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you

In [35]:
query = "what is the price of BNB?"
langchain.debug = True
chain({"question":query}, return_only_outputs = True)

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of BNB?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Mark Cuban Predicts World's First 'Trillionaire' Won't Be Musk Or Bezos — It'll Be Someone Who 'Masters AI' And Applies It In Brand New WaysBenzinga\n\nFord sales surge 6.8% powered by hybrids and EVsYahoo Finance\n\nWhy Apollo has a leg up in Paramount deal: AnalystYahoo Finance Video\n\nWhy are home prices so high?Yahoo Personal Finance\n\nDollar Tree Is Raising Prices: 9 Products To Buy Now Before They IncreaseGOBankingRates\n\nInvestors Paid $65,000 For A Deserted Florida Island. Now It's A Paradise Worth $14 MillionBenzinga\n\nLevi Strauss stock jumps af

{'answer': " I don't know.\n",
 'sources': 'Unable to provide due to lack of relevant content.'}